 #Detección de Valores Perdidos y de Valores Duplicados

Importamos el paquete que vayamos a necesitar

In [1]:
 import pandas as pd
 import numpy as np 
 import matplotlib.pyplot as pyplot
 import os



Importamos los datos, utilizando el parámetro chunksize, para manejar datasets grandes

In [2]:

#cargo el dataset
df = pd.read_csv(r"C:\Users\lucia\Desktop\5º\TFG\prepared OULAD dataset\data_final.csv", sep= ';')
df.columns = df.columns.str.strip()  # Elimina espacios en los nombres de las columnas


In [3]:
# Mostrar las primeras filas para verificar
print(df.head())

   Unnamed: 0  id_student  date  homepage  oucontent  subpage  url  forumng  \
0           0       11391     0      10.0       21.0      0.0  0.0     15.0   
1           1       11391     1       9.0      102.0      8.0  1.0      3.0   
2           2       11391     2       3.0        0.0      0.0  0.0      0.0   
3           3       11391     3       0.0        0.0      0.0  0.0      0.0   
4           4       11391     4       0.0        0.0      0.0  0.0      0.0   

   resource  repeatactivity  ...  imd_band_50-60%  imd_band_60-70%  \
0       3.0             0.0  ...                0                0   
1       4.0             0.0  ...                0                0   
2       1.0             0.0  ...                0                0   
3       0.0             0.0  ...                0                0   
4       0.0             0.0  ...                0                0   

   imd_band_70-80%  imd_band_80-90%  imd_band_90-100%  age_band_0-35  \
0                0              

In [4]:
# Verificar el número de filas y columnas
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {len(df.columns)}")



Número de filas: 1048575
Número de columnas: 73


In [5]:
#print("Columnas después de corregir la lectura:")
print(df.columns.tolist())


['Unnamed: 0', 'id_student', 'date', 'homepage', 'oucontent', 'subpage', 'url', 'forumng', 'resource', 'repeatactivity', 'glossary', 'dataplus', 'oucollaborate', 'htmlactivity', 'questionnaire', 'dualpane', 'quiz', 'externalquiz', 'page', 'folder', 'ouwiki', 'sharedsubpage', 'ouelluminate', 'num_of_prev_attempts', 'studied_credits', 'final_result', 'code_module_AAA', 'code_module_BBB', 'code_module_CCC', 'code_module_DDD', 'code_module_EEE', 'code_module_FFF', 'code_module_GGG', 'code_presentation_2013B', 'code_presentation_2013J', 'code_presentation_2014B', 'code_presentation_2014J', 'gender_F', 'gender_M', 'region_East Anglian Region', 'region_East Midlands Region', 'region_Ireland', 'region_London Region', 'region_North Region', 'region_North Western Region', 'region_Scotland', 'region_South East Region', 'region_South Region', 'region_South West Region', 'region_Wales', 'region_West Midlands Region', 'region_Yorkshire Region', 'highest_education_A Level or Equivalent', 'highest_edu

In [6]:
# Eliminar la columna 'Unnamed: 0' si no es necesaria
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [7]:
print(df.columns)

Index(['id_student', 'date', 'homepage', 'oucontent', 'subpage', 'url',
       'forumng', 'resource', 'repeatactivity', 'glossary', 'dataplus',
       'oucollaborate', 'htmlactivity', 'questionnaire', 'dualpane', 'quiz',
       'externalquiz', 'page', 'folder', 'ouwiki', 'sharedsubpage',
       'ouelluminate', 'num_of_prev_attempts', 'studied_credits',
       'final_result', 'code_module_AAA', 'code_module_BBB', 'code_module_CCC',
       'code_module_DDD', 'code_module_EEE', 'code_module_FFF',
       'code_module_GGG', 'code_presentation_2013B', 'code_presentation_2013J',
       'code_presentation_2014B', 'code_presentation_2014J', 'gender_F',
       'gender_M', 'region_East Anglian Region', 'region_East Midlands Region',
       'region_Ireland', 'region_London Region', 'region_North Region',
       'region_North Western Region', 'region_Scotland',
       'region_South East Region', 'region_South Region',
       'region_South West Region', 'region_Wales',
       'region_West Midlands R

Configuramos para que se muestren todas las columnas
pd.set_option('display.max_rows', None)
Vemos el tipo de variables que se han cargado
estudiantes.head(100)

In [8]:
columnas_actividad = [
    "ouelluminate", "sharedsubpage", "ouwiki", "folder", "page", "externalquiz",
    "quiz", "dualpane", "questionnaire", "htmlactivity", "oucollaborate",
    "dataplus", "glossary", "repeatactivity", "resource", "forumng", "url",
    "subpage", "oucontent", "homepage"
]


In [9]:
#verifica la existencia de estas columnas
columnas_actividad_validas = [col for col in columnas_actividad if col in df.columns]

Lista de columnas de actividad

In [10]:
# Mostrar las columnas faltantes
columnas_faltantes = set(columnas_actividad) - set(columnas_actividad_validas)
if columnas_faltantes:
    print("Columnas faltantes en el DataFrame:", columnas_faltantes)

In [11]:
#convertimos la columna "final_result" en la columna binaria: 1 = abandono, 0 = no abandono
df['final_result'] = df['final_result'].str.strip().str.lower() #eliminando espacios y mayusculas
df['final_result'] = df['final_result'].apply(lambda x: 1 if x == 'withdrawn' else 0)


In [12]:
# Ordenar por id_student y date en orden descendente (de más reciente a más antigua)
df = df.sort_values(by=['id_student', 'date'], ascending=[True, False])


En las columnas que haya abandono (1), pero no haya ninguna actividad, es decir todas las columnas de actividad para ese alumno sean 0, tiene que alamcenar la columna 0

In [13]:
# Función para seleccionar la fila adecuada por estudiante
def seleccionar_fila(grupo):
    if grupo['final_result'].iloc[0] == 0:
        fila_269 = grupo[grupo['date'] == 269]
        if not fila_269.empty:
            return fila_269.iloc[0]
        else:
            return grupo.iloc[0]  # Si no hay date=269, se toma la más reciente
    else:
        grupo_con_actividad = grupo[(grupo[columnas_actividad_validas] > 0.0).any(axis=1)]
        if not grupo_con_actividad.empty:
            return grupo_con_actividad.iloc[0]
        else: #no tiene actividad, por tanto devolvemos la fila con date=0
            fila_0 = grupo[grupo['date'] == 0]
            if not fila_0.empty:
                return fila_0.iloc[0]  # Por seguridad, aunque no debería pasar

            else:
                return grupo.iloc[-1]

In [14]:
# Eliminar duplicados quedándonos con la fila más reciente por cada id_student (aplica la funcion aimplementada anteriormente)
df_final = df.groupby('id_student', group_keys=False).apply(seleccionar_fila)


C:\Users\lucia\AppData\Local\Temp\ipykernel_22820\3347431866.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df.groupby('id_student', group_keys=False).apply(seleccionar_fila)


In [15]:
# Mostrar el número de filas antes y después
print(f"Filas antes de filtrar: {len(df)}")
#print(f"Filas después de filtrar: {len(df_filtrado)}")
print(f"Filas después de eliminar duplicados: {len(df_final)}")

Filas antes de filtrar: 1048575
Filas después de eliminar duplicados: 3810


In [16]:
# Convertimos todas las columnas a enteros, excepto las de actividad 
columnas_no_convertir = set(columnas_actividad_validas)

for col in df_final.columns:
    if col not in columnas_no_convertir:
        df_final[col] = df_final[col].astype(int)

Unificar las columnas binarias que sean de 2 posibles valores, borrando una de las dos columnas y cambiandole a un nombre general (gender y disability)

In [17]:
# Columnas a eliminar: todas las de actividad, 'gender_M' y 'disability_Y'
columnas_a_conservar = ['id_student', 'date', 'final_result', 'gender_F', 'disability_N']

#verificamos que existen anets de borrarlas
columnas_a_eliminar = [col for col in df_final.columns if col not in columnas_a_conservar]

#las borramos
df_final = df_final.drop(columns=columnas_a_eliminar)

print(f"Columnas eliminadas: {columnas_a_eliminar}")
print(f"Columnas restantes: {df_final.columns.tolist()}")


Columnas eliminadas: ['homepage', 'oucontent', 'subpage', 'url', 'forumng', 'resource', 'repeatactivity', 'glossary', 'dataplus', 'oucollaborate', 'htmlactivity', 'questionnaire', 'dualpane', 'quiz', 'externalquiz', 'page', 'folder', 'ouwiki', 'sharedsubpage', 'ouelluminate', 'num_of_prev_attempts', 'studied_credits', 'code_module_AAA', 'code_module_BBB', 'code_module_CCC', 'code_module_DDD', 'code_module_EEE', 'code_module_FFF', 'code_module_GGG', 'code_presentation_2013B', 'code_presentation_2013J', 'code_presentation_2014B', 'code_presentation_2014J', 'gender_M', 'region_East Anglian Region', 'region_East Midlands Region', 'region_Ireland', 'region_London Region', 'region_North Region', 'region_North Western Region', 'region_Scotland', 'region_South East Region', 'region_South Region', 'region_South West Region', 'region_Wales', 'region_West Midlands Region', 'region_Yorkshire Region', 'highest_education_A Level or Equivalent', 'highest_education_HE Qualification', 'highest_educatio

In [20]:
df_final.to_csv("dataset_limpio.csv", index=False, sep=';')

In [21]:
# Volver a cargar el archivo con el delimitador correcto (;)
df1 = pd.read_csv(r"C:\Users\lucia\Desktop\5º\TFG\prepared OULAD dataset\dataset_limpio.csv", sep = ";", engine="python")
print(f"Número de filas: {df_final.shape[0]}")
print(f"Número de columnas: {len(df_final.columns)}")

Número de filas: 3810
Número de columnas: 5
